In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import operator
import math
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import itertools
import plotly.offline as py
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium import *
import plotly.graph_objects as go
%matplotlib inline

In [75]:
location_csv = "/home/hp/Desktop/College/Sem5/DA/Project/Restaurant-Rating-Prediction/my_location.csv"
infzomato_csv = "/home/hp/Desktop/College/Sem5/DA/Project/infzomato.csv"

In [76]:
#df = pd.read_csv("/home/hp/Desktop/College/Sem5/DA/Project/zomato-bangalore-restaurants/zomato.csv")
mylist = []

for chunk in  pd.read_csv("/home/hp/Desktop/College/Sem5/DA/Project/newzomato.csv", sep=',', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
df = big_data

del mylist


In [77]:
#Nans values are filled in with median of the particular
df['rate'].fillna((df['rate'].median()), inplace=True)
df['approx_cost(for two people)'].fillna((df['approx_cost(for two people)'].median()), inplace=True)

In [78]:
transformed_df = df.copy()
all_cus = list(df['Cuisine1'])
temp = set(all_cus)
k = []

for i in temp:
    k.append((i,all_cus.count(i)))
    
for i in k:
    if(i[1]<1100):
        transformed_df = transformed_df[transformed_df['Cuisine1']!=i[0]]

In [79]:
#factorize the categorical variables 
transformed_df['new_Cuisine1'], cus_label = pd.factorize(transformed_df['Cuisine1'], sort=False)
transformed_df['new_location'], loc_label = pd.factorize(transformed_df['location'], sort=False)
transformed_df['new_online_order'], on_label = pd.factorize(transformed_df['online_order'], sort=False)
transformed_df['new_book_table'], book_label = pd.factorize(transformed_df['book_table'], sort=False)

to_drop = ['Cuisine1',
           'Cuisine2',
           'rest1',
           'listed_in(type)',
           'location',
           'online_order',
           'book_table',
           'name']
transformed_df.drop(to_drop, inplace=True, axis=1)
transformed_df.dropna(how='any', inplace=True)


Decision tree regressor model parameters selecting based on feature importance in the previous file

In [80]:
train, test = train_test_split(transformed_df, test_size=0.2)

In [81]:
train_x = train.loc[:, train.columns != 'rate']
train_y = train.loc[:, train.columns == 'rate']
test_x = test.loc[:, test.columns != 'rate']
test_y = test.loc[:, test.columns == 'rate']

In [82]:
#The count of certain cuisines are very less due to which the model can be biased.
#All cuisines with the count < 1000 are dropped 
temp = []
cus = list(set(list(train_x['new_Cuisine1'])))
for i in cus:
    temp.append(cus_label[i])
    
all_cus = list(df['Cuisine1'])
k = []

for i in temp:
    k.append((i,all_cus.count(i)))
    
for i in k:
    if(i[1]<1000):
        transformed_df = transformed_df[transformed_df['Cuisine1']!=i[0]]


In [83]:
regressor = DecisionTreeRegressor(random_state = 0)

In [84]:
regressor.fit(train_x, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [85]:
y_pred = list(regressor.predict(test_x))

In [86]:
y_pred = np.around(y_pred,decimals=1)
test_y = list(test_y['rate'])
y_pred = list(y_pred)
acc = 0
for i in range(len(y_pred)):
    if(round((y_pred[i] - test_y[i]),1)<0.1):
        acc = acc + 1
print(acc)
print(acc/len(y_pred))

6544
0.7363564757510971


we would like to find the attributes of restraunt in each location so as to miximize their rating

In [87]:
#Generating all possible combinations of attributes for a particular location 
apprx = [200,400,600,800,1000,1200,1400,1600,1800,2000]
cus = list(set(list(train_x['new_Cuisine1'])))
online = [0,1]
book = [0,1]
infered_results = []
for i in list(set(list(train_x['new_location']))):
    m  = [0,[]]
    loc = [i]
    k = [apprx,cus,loc,online,book]
    for element in  itertools.product(*k):
        p = round(list(regressor.predict([element]))[0],2)
        if(p>m[0]):
            m[0] = p
            m[1] = [element[0],cus_label[element[1]],loc_label[element[2]],on_label[element[3]],book_label[element[4]]]
    infered_results.append(m)

In [88]:
#Customers prefer restaurants that have online ordering facility over those that don't

infered_on = 0 
for i in infered_results:
    if i[1][3] == 'Yes':
        infered_on = infered_on + 1
labels = ['Yes','No']
values = [infered_on, len(infered_results)-infered_on]
colors = ['#00203FFF', '#ADEFD1FF']

trace=go.Pie(labels=labels,values=values,textinfo="value",
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
layout=go.Layout(title="Accepting vs not accepting online ordering",width=500,height=500)
fig=go.Figure(data=[trace],layout=layout)
py.iplot(fig, filename='pie_chart_subplots')

In [89]:
#Restraunts having table booking facility tend to attract more customers and maximize their rating
infered_book = 0 
for i in infered_results:
    if i[1][4] == 'Yes':
        infered_book = infered_book + 1
labels = ['Yes','No']
values = [infered_book, len(infered_results)-infered_book]
colors = ['#101820FF', '#FEE715FF']

trace=go.Pie(labels=labels,values=values,textinfo="value",
            marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))
layout=go.Layout(title="Accepting vs not accepting online booking",width=500,height=500)
fig=go.Figure(data=[trace],layout=layout)
py.iplot(fig, filename='pie_chart_subplots')
    

In [90]:
#Which is the most popular Cuisine that miximized the rating?
infered_cus = {}
for i in infered_results:
    if not i[1][1] in infered_cus:
        infered_cus[i[1][1]] = 1
    else:
        infered_cus[i[1][1]] = infered_cus[i[1][1]] + 1

print("The two most popular cuisines are")
print("1."+sorted(infered_cus,key = lambda x: -infered_cus[x])[0]+"\n2."+sorted(infered_cus,key = lambda x: -infered_cus[x])[1])

The two most popular cuisines are
1.North Indian
2.Cafe


In [91]:
# What is the average price people would pay if all restaurant's ratings were maximized
total_price = 0
for i in infered_results:
    total_price = total_price + i[1][0]

print("Average price per restaurant:")
print((int(total_price/len(infered_results))//100)*100)

Average price per restaurant:
1300


In [92]:
#Code to generate the infered results csv file consisting of locations along with the attributes
#that maximized the rating for restraunts in that location
"""
inf = []
for i in range(len(infered_results)):
    inf.append([infered_results[i][0]])
    inf[i].extend(infered_results[i][1])

inf_df = pd.DataFrame(inf, columns =['Rating', 'Approx. cost for 2', 'Cuisine','location','online_order',
        'online_table_book'], dtype = float) 
        
export_csv = inf_df.to_csv ('/home/hp/Desktop/College/Sem5/DA/Project/Restaurant-Rating-Prediction/infzomato.csv', index = None, header=True)
"""

"\ninf = []\nfor i in range(len(infered_results)):\n    inf.append([infered_results[i][0]])\n    inf[i].extend(infered_results[i][1])\n\ninf_df = pd.DataFrame(inf, columns =['Rating', 'Approx. cost for 2', 'Cuisine','location','online_order',\n        'online_table_book'], dtype = float) \n        \nexport_csv = inf_df.to_csv ('/home/hp/Desktop/College/Sem5/DA/Project/Restaurant-Rating-Prediction/infzomato.csv', index = None, header=True)\n"

In [93]:
mylist =[]
for chunk in  pd.read_csv(infzomato_csv, sep=',', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
inf_df = big_data

In [94]:
loc_df

,,<<<<<<< HEAD
location,latitude,longitude
Banashankari,12.9152208,77.573598
Basavanagudi,12.9417261,77.5755021
Mysore Road,12.9104361,77.4836681
Jayanagar,12.9292731,77.5824229
Kumaraswamy Layout,12.9068169,77.5635247362134
Rajarajeshwari Nagar,12.9274278,77.5153722
Vijay Nagar,12.9718886,77.5457888388713
Uttarahalli,12.9055682,77.5455438
JP Nagar,12.9072515,77.5782713


In [95]:
#Code to obtain the latitude and logitube for each location in infered results
#exporting it into my_location.csv file
"""
locations = {}
for i in infered_results:
    locations[i[1][2]] = []
    
geolocator=Nominatim(user_agent="app")
for i in locations:
    locator = geolocator.geocode("Bangalore "+i)
    locations[i].append((locator.latitude,locator.longitude))
    
my_locs = []
for k in locations:
    my_locs.append([k,locations[k][0][0],locations[k][0][1]])
    
loc_df = pd.DataFrame(my_locs, columns =['location', 'latitude', 'longitude'],  dtype = float) 

export_csv = loc_df.to_csv ('C:/Users/sparsha/Documents/SEM 5/DA Project/zomato/my_location.csv', index = None, header=True)
"""

'\nlocations = {}\nfor i in infered_results:\n    locations[i[1][2]] = []\n    \ngeolocator=Nominatim(user_agent="app")\nfor i in locations:\n    locator = geolocator.geocode("Bangalore "+i)\n    locations[i].append((locator.latitude,locator.longitude))\n    \nmy_locs = []\nfor k in locations:\n    my_locs.append([k,locations[k][0][0],locations[k][0][1]])\n    \nloc_df = pd.DataFrame(my_locs, columns =[\'location\', \'latitude\', \'longitude\'],  dtype = float) \n\nexport_csv = loc_df.to_csv (\'C:/Users/sparsha/Documents/SEM 5/DA Project/zomato/my_location.csv\', index = None, header=True)\n'

In [96]:
mylist =[]
for chunk in  pd.read_csv(location_csv, sep=',', chunksize=20000):
    mylist.append(chunk)

big_data = pd.concat(mylist, axis= 0)
loc_df = big_data

In [97]:
#dataframe consisting of all the details including location co-ordinates
result = pd.concat([loc_df, inf_df], axis=1, join='inner')

In [98]:
# random number genetator model round(random.uniform(0.25,0.5),2)*10
diff=['0.0', '0.1', '0.2']

fig = go.Figure(data=[go.Bar(name='Linear Regression', x=diff, y=[0.13,0.41,0.59]),
    go.Bar(name='Decision Trees', x=diff, y=[0.75,0.82,0.87]),
    go.Bar(name='Random Number generator', x=diff, y=[0.043, 0.11, 0.20])
])
# Change the bar mode
fig.update_xaxes(title_text='Difference between actual and predicted')
fig.update_yaxes(title_text='Accuracy')
fig.update_layout(barmode='group')
fig.show()

In [99]:
feature_group = FeatureGroup(name='North Indian')
feature_group2 = FeatureGroup(name='Cafe')
feature_group3 = FeatureGroup(name='South Indian')
feature_group4 = FeatureGroup(name='Desserts')
feature_group5 = FeatureGroup(name='Chinese')

In [100]:
map_hooray = folium.Map(location=[12.9716,77.5946],
                        tiles = "OpenStreetMap",
                        zoom_start = 12.4)
colors = {'North Indian':'red','South Indian':'blue','Desserts':'brown','Chinese':'black','Cafe':'green'}
for row in result.iterrows():
    if(row[1][5] == 'North Indian'):
        folium.CircleMarker([float(row[1][1]),float(row[1][2])],
                        radius=5,
                        popup="loc:"+row[1][0]+"\ncuisine:"+row[1][5],
                        color=colors[row[1][5]],
                        fill = True,
                        ).add_to(feature_group)
    if(row[1][5] == 'Cafe'):
        folium.CircleMarker([float(row[1][1]),float(row[1][2])],
                        radius=5,
                        popup="loc:"+row[1][0]+"\ncuisine:"+row[1][5],
                        color=colors[row[1][5]],
                        fill = True,
                        ).add_to(feature_group2)
    if(row[1][5] == 'South Indian'):
        folium.CircleMarker([float(row[1][1]),float(row[1][2])],
                        radius=5,
                        popup="loc:"+row[1][0]+"\ncuisine:"+row[1][5],
                        color=colors[row[1][5]],
                        fill = True,
                        ).add_to(feature_group3)
    if(row[1][5] == 'Desserts'):
        folium.CircleMarker([float(row[1][1]),float(row[1][2])],
                        radius=5,
                        popup="loc:"+row[1][0]+"\ncuisine:"+row[1][5],
                        color=colors[row[1][5]],
                        fill = True,
                        ).add_to(feature_group4)
    if(row[1][5] == 'Chinese'):
        folium.CircleMarker([float(row[1][1]),float(row[1][2])],
                        radius=5,
                        popup="loc:"+row[1][0]+"\ncuisine:"+row[1][5],
                        color=colors[row[1][5]],
                        fill = True,
                        ).add_to(feature_group5)

map_hooray.add_child(feature_group)
map_hooray.add_child(feature_group2)
map_hooray.add_child(feature_group3)
map_hooray.add_child(feature_group4)
map_hooray.add_child(feature_group5)
map_hooray.add_child(folium.map.LayerControl('topright', collapsed=False))


Possiable values for location are shown in the map
eg: BTM, HSR, JP Nagar, Hennur etc.

In [102]:
#Driver Code which recommeds optimal price for 2, Cuisine, Allow online order Allow online booking in order to maximize rating
k = input("Enter Location: ")
for i in infered_results:
    if k == i[1][2]:
        print("Recommended:\nPrice for two:%s\nCuisine:%s\nAllow online order:%s\nAllow Online Booking:%s\n"%(str(i[1][0]),i[1][1],i[1][3],i[1][4]))
        print("Expected rating "+str(i[0]))

Enter Location: HSR
Recommended:
Price for two:1000
Cuisine:North Indian
Allow online order:No
Allow Online Booking:Yes

Expected rating 4.7


In [103]:
cus_d = {}
for i in list(set(list(train_x['new_Cuisine1']))):
    cus_d[cus_label[i]] = i
    
on_d = {}
for i in list(set(list(train_x['new_online_order']))):
    on_d[on_label[i]]=i

book_d = {}
for i in list(set(list(train_x['new_book_table']))):
    book_d[book_label[i]]=i

Possiable values for Cuisine : North Indian,Cafe,Continental,Chinese,Fast Food,South Indian,Desserts,Biryani
Possiable values for online order,book table : Yes, No

In [104]:
# Given optimal price for 2, Cuisine, Allow online order and book table suggest a location to maximize profit
p = int(input("Price for 2: "))
c = input("Cuisine: ")
o = input("Online: ")
b = input("Booking: ")

c = cus_d[c]
o = on_d[o]
b = book_d[b]

m = [0]
for i in list(set(list(train_x['new_location']))):
    try:
        k = round(list(regressor.predict([[p,c,i,o,b]]))[0],2)
        if k>m[0]:
            m = [k]
            m.append(([p,cus_label[int(c)],loc_label[int(i)],on_label[int(o)],book_label[int(b)]]))
    except:
        print("Model doesnt support inputs")

print("Suggested Location: "+str(m[1][2]))
print("Expected Rating "+str(m[0]))

Price for 2: 


ValueError: invalid literal for int() with base 10: ''